In [1]:
# import packages
import sys
import ee
import numpy as np
import pandas as pd
import time

In [2]:
# Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [3]:
# load in management units as feature collection
management_units = ee.FeatureCollection('projects/resource-watch-gee/LocalDatasets/GCBR/loc_gcbr_013_rw1_management_units')
management_units_properties = management_units.first().propertyNames().remove('system:index')
print('Management Unit Properties: ',management_units_properties.getInfo())


Management Unit Properties:  ['Ha', 'Unit_TW']


In [4]:
# Get monthly total burn area from MODIS Burn Area Dataset
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD64A1

# define output name and folder for Google Drive
output_name = 'GCBR_MODIS_burn_area'
output_folder = 'DOB_Ecology/GCBR'

# load in monthly Modis burn area collection:
# each image represents the burn area for a month, with pixels encoded with the julian date of the fire
modis_burn_area_collection = ee.ImageCollection("MODIS/006/MCD64A1")

# get projection information
modis_projection = modis_burn_area_collection.first().projection().getInfo()
modis_projection_gee = modis_burn_area_collection.first().projection()
modis_crs = modis_projection.get('crs')
modis_crsTransform = modis_projection.get('transform')

# select burn date band from each image
modis_burn_date_collection = modis_burn_area_collection.map(lambda x: ee.Image(x).select('BurnDate'))
# convert from image collection to multiband image, with each band representing one month 
modis_burn_date_image = modis_burn_date_collection.toBands()

# convert from julian burn date to burn area by multiplying by pixel area and converting from square meters to square hectares
modis_burn_area_image = modis_burn_date_image.gt(ee.Image.constant(0)).multiply(ee.Image.pixelArea()).multiply(ee.Image.constant(1e-4))

# calculate the sum of the burn area for each management unit
def calculate_burn_area(feature_collection):
    sum_burn_area = modis_burn_area_image.reduceRegions(feature_collection, 
                                                          ee.Reducer.sum(), 
                                                          crs=modis_crs, crsTransform=modis_crsTransform, 
                                                          tileScale=8)
    #Drop geometry information
    sum_burn_area = sum_burn_area.map(lambda x: x.select(x.propertyNames(),retainGeometry=False))
    #Return
    return ee.FeatureCollection(sum_burn_area)

# Transpose results, this is so the results can be easily manipulated with our 
# custom visualization tool in Resource Watch. Converting from wide format,
# one row for each region with columns for each time-step of data, to long
# format, having fewer columns including "date" and "value" and a row
# for each time-step
def convert_wide_to_long(feature):
    def select_band(band_name):
        band_name = ee.String(band_name)
        out_feature = ee.Feature(None,{})
        out_feature = out_feature.copyProperties(feature, management_units_properties)
        
        value = 0
        value = feature.get(band_name)
        
        band_name_split = band_name.split('_')
        year = ee.Number.parse(band_name_split.get(0))
        month = ee.Number.parse(band_name_split.get(1))
        date = ee.Date.fromYMD(year, month, 1)
        date = date.format('YYYY-MM-dd')
        
        out_feature = out_feature.set('Value',value,'Year',year,'Month',month,'Date',date,'Unit','Hectares')
        return out_feature
    
    output_collection = modis_burn_area_image.bandNames().map(select_band)
    return ee.FeatureCollection(output_collection)


# run functions
output = calculate_burn_area(management_units)
output = output.map(convert_wide_to_long).flatten()

#Export to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection = output, 
    description = output_name, 
    fileNamePrefix = output_name,
    folder = output_folder)

export_results_task.start()


In [5]:
# Get monthly total fire brightness from FIRMS: Fire Information for Resource Management System dataset
# https://developers.google.com/earth-engine/datasets/catalog/FIRMS

# Define output name and folder for the exported data
output_name = 'GCBR_FIRMS_fire_brightness'
output_folder = 'DOB_Ecology/GCBR'

# Load the FIRMS image collection, which shows the brightness of fires
firms_collection = ee.ImageCollection("FIRMS")

# Get projection information from the first image in the FIRMS collection
firms_projection = firms_collection.first().projection().getInfo()
firms_projection_gee = firms_collection.first().projection()
firms_crs = firms_projection.get('crs')
firms_crsTransform = firms_projection.get('transform')

# Select only the brightness band (T21) from images in the collection
firms_brightness_collection = firms_collection.map(lambda x: ee.Image(x).select(['T21']))

# Define lists of months and years to loop over
months = ee.List([str(x) for x in np.arange(1, 13)])
years = ee.List([str(x) for x in np.arange(2000, 2021)])

# Define a function to filter the FIRMS brightness collection for a specific year and month, calculate the sum, and set the system:index property
def filter_year(y):
    def filter_month(m):
        year = ee.Number.parse(y)
        month = ee.Number.parse(m)
        new_image = firms_brightness_collection.filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(month, month, 'month')).sum()
        new_image = new_image.set('system:index', ee.String(y).cat(ee.String('_')).cat(ee.String(m)))
        return new_image
    return months.map(filter_month)

# Create a new ImageCollection by mapping the filter_year function over the years and months
firms_brightness_collection = ee.ImageCollection.fromImages(years.map(filter_year).flatten())
firms_brightness_image = firms_brightness_collection.toBands()

# Define a function to calculate the total fire brightness for each feature in the input feature collection
def calculate_fire_brightness(feature_collection):
    # Calculate the total fire brightness within each feature (geometry) in the feature_collection
    sum_brightness = firms_brightness_image.reduceRegions(feature_collection, 
                                                         ee.Reducer.sum(), 
                                                         crs=firms_crs, crsTransform=firms_crsTransform, 
                                                         tileScale=8)
    # Drop geometry information to keep only the properties
    sum_brightness = sum_brightness.map(lambda x: x.select(x.propertyNames(), retainGeometry=False))
    # Return the feature collection with total fire brightness information
    return ee.FeatureCollection(sum_brightness)

# Define a function to convert the wide-format output to long-format
def convert_wide_to_long(feature):
    # Define a function to select the fire brightness value for a specific band (date)
    def select_band(band_name):
        band_name = ee.String(band_name)
        out_feature = ee.Feature(None, {})
        out_feature = out_feature.copyProperties(feature, management_units_properties)

        value = 0
        value = feature.get(band_name)

        band_name_split = band_name.split('_')
        year = ee.Number.parse(band_name_split.get(0))
        month = ee.Number.parse(band_name_split.get(1))
        day = 1
        date = ee.Date.fromYMD(year, month, day)
        date = date.format('YYYY-MM-dd')

        out_feature = out_feature.set('Value', value, 'Year', year, 'Month', month, 'Day', day, 'Date', date, 'Unit', 'Kelvin')
        return out_feature

    # Map the select_band function over the band names (dates) to create a new FeatureCollection
    output_collection = firms_brightness_image.bandNames().map(select_band)
    return ee.FeatureCollection(output_collection)


# Run functions
output = calculate_fire_brightness(management_units)
output = output.map(convert_wide_to_long).flatten()

# Export the results to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection=output,
    description=output_name,
    fileNamePrefix=output_name,
    folder=output_folder)

# Start the export task
export_results_task.start()



In [6]:
# Get the annual average temperature from NOAA's GHCN CAMS temperature dataset
#https://psl.noaa.gov/data/gridded/data.ghcncams.html
    
# define output name and folder for Google Drive
output_name = 'GCBR_GHCN_CAMS_average_temperature'
output_folder = 'DOB_Ecology/GCBR'

# load in GHCN_CAMS temperature dataset, which is processed by our preprocessing script:
# https://github.com/resource-watch/blog-analysis/tree/master/req_016_facebook_average_surface_temperature
temperature_collection = ee.ImageCollection('projects/resource-watch-gee/Facebook/TemperatureAnalysis/GHCN_CAMS')

# get projection information
temperature_projection = temperature_collection.first().projection().getInfo()
temperature_projection_gee = temperature_collection.first().projection()
temperature_crs = temperature_projection.get('crs')
temperature_crsTransform = temperature_projection.get('transform')

# convert image collection of annual average temperature to multiband image
temperature_image = temperature_collection.toBands()
# convert from Kelvin to Celsius by subtracting constant
temperature_image = temperature_image.add(ee.Image.constant(-273.15))

def calculate_average_temperature(feature_collection):
    #Calculate average temperature in management areas
    average_temperature = temperature_image.reduceRegions(feature_collection, 
                                                          ee.Reducer.mean(), 
                                                          crs=temperature_crs, crsTransform=temperature_crsTransform, 
                                                          tileScale=8)
    #Drop geometry information
    average_temperature = average_temperature.map(lambda x: x.select(x.propertyNames(),retainGeometry=False))
    #Return
    return ee.FeatureCollection(average_temperature)

# transpose results
def convert_wide_to_long(feature):
    def select_band(band_name):
        band_name = ee.String(band_name)
        out_feature = ee.Feature(None,{})
        out_feature = out_feature.copyProperties(feature, management_units_properties)
        
        value = 0
        value = feature.get(band_name)

        band_name_split = band_name.split('_')
        year = ee.Number.parse(band_name_split.get(0))
        month = ee.Number.parse(band_name_split.get(1))
        day = ee.Number.parse(band_name_split.get(2))
        date = ee.Date.fromYMD(year, month, day)
        date = date.format('YYYY-MM-dd')
        
        out_feature = out_feature.set('Value',value,'Year',year,'Month',month,'Day',day,'Date',date,'Unit','Celsius')
        return out_feature
    
    output_collection = temperature_image.bandNames().map(select_band)
    return ee.FeatureCollection(output_collection)


output = calculate_average_temperature(management_units)
output = output.map(convert_wide_to_long).flatten()

#Export to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection = output, 
    description = output_name, 
    fileNamePrefix = output_name,
    folder = output_folder)

export_results_task.start()


In [7]:
# Get monthly NDVI from the MODIS satellite
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13Q1

# Define output name and folder for the exported data
output_name = 'GCBR_MODIS_average_NDVI'
output_folder = 'DOB_Ecology/GCBR'

# Load the NDVI data from the MODIS/006/MOD13Q1 dataset
ndvi_collection = ee.ImageCollection('MODIS/006/MOD13Q1')

# Get projection information from the first image in the NDVI collection
ndvi_projection = ndvi_collection.first().projection().getInfo()
ndvi_projection_gee = ndvi_collection.first().projection()
ndvi_crs = ndvi_projection.get('crs')
ndvi_crsTransform = ndvi_projection.get('transform')

# Select only the NDVI band from the NDVI collection and convert to a multiband image
ndvi_collection = ndvi_collection.map(lambda x: ee.Image(x).select('NDVI'))
ndvi_image = ndvi_collection.toBands()

# Define a function to calculate the average NDVI for each feature in the input feature collection
def calculate_average_ndvi(feature_collection):
    # Calculate the average NDVI within each feature (geometry) in the feature_collection
    average_ndvi = ndvi_image.reduceRegions(feature_collection, 
                                            ee.Reducer.mean(), 
                                            crs=ndvi_crs, crsTransform=ndvi_crsTransform, 
                                            tileScale=8)
    # Drop geometry information to keep only the properties
    average_ndvi = average_ndvi.map(lambda x: x.select(x.propertyNames(), retainGeometry=False))
    # Return the feature collection with average NDVI information
    return ee.FeatureCollection(average_ndvi)

# Define a function to convert the wide-format output to long-format
def convert_wide_to_long(feature):
    # Define a function to select the NDVI value for a specific band (date)
    def select_band(band_name):
        band_name = ee.String(band_name)
        out_feature = ee.Feature(None, {})
        out_feature = out_feature.copyProperties(feature, management_units_properties)

        value = 0
        value = feature.get(band_name)

        band_name_split = band_name.split('_')
        year = ee.Number.parse(band_name_split.get(0))
        month = ee.Number.parse(band_name_split.get(1))
        day = ee.Number.parse(band_name_split.get(2))
        date = ee.Date.fromYMD(year, month, day)
        date = date.format('YYYY-MM-dd')

        out_feature = out_feature.set('Value', value, 'Year', year, 'Month', month, 'Day', day, 'Date', date, 'Unit', 'None')
        return out_feature

    # Map the select_band function over the band names (dates) to create a new FeatureCollection
    output_collection = ndvi_image.bandNames().map(select_band)
    return ee.FeatureCollection(output_collection)

# Calculate average NDVI for each management unit in the input feature collection
output = calculate_average_ndvi(management_units)

# Convert the output from wide to long format
output = output.map(convert_wide_to_long).flatten()

# Export the results to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection=output,
    description=output_name,
    fileNamePrefix=output_name,
    folder=output_folder)

# Start the export task
export_results_task.start()


In [8]:
# Get monthly Leaf Area Index (LAI) and Fraction of Absorbed Photosynthetically Active Radiation (FAPAR) from NOAA
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_CDR_AVHRR_LAI_FAPAR_V5

# Define output name and folder for the exported data
output_name = 'GCBR_AVHRR_average_LAI_FAPAR'
output_folder = 'DOB_Ecology/GCBR'

# Load the LAI and FAPAR collection from NOAA
lai_collection = ee.ImageCollection("NOAA/CDR/AVHRR/LAI_FAPAR/V5")

# Get projection information from the first image in the LAI collection
lai_projection = lai_collection.first().projection().getInfo()
lai_projection_gee = lai_collection.first().projection()
lai_crs = lai_projection.get('crs')
lai_crsTransform = lai_projection.get('transform')

# Select only LAI and FAPAR bands from the LAI collection
lai_collection = lai_collection.map(lambda x: ee.Image(x).select('LAI', 'FAPAR'))

# Define lists of months and years to loop over
months = ee.List([str(x) for x in np.arange(1, 13)])
years = ee.List([str(x) for x in np.arange(1982, 2021)])

# Define a function to filter the LAI collection for a specific year and month, calculate the mean, and set the system:index property
def filter_year(y):
    def filter_month(m):
        year = ee.Number.parse(y)
        month = ee.Number.parse(m)
        new_image = lai_collection.filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(month, month, 'month')).mean()
        new_image = new_image.set('system:index', ee.String(y).cat(ee.String('_')).cat(ee.String(m)))
        return new_image
    return months.map(filter_month)

# Create a new ImageCollection by mapping the filter_year function over the years and months
lai_collection = ee.ImageCollection.fromImages(years.map(filter_year).flatten())

# Convert the ImageCollection to a single multiband image
lai_image = lai_collection.toBands()

# Define a function to calculate the average LAI and FAPAR for each feature in the input feature collection
def calculate_average_lai(feature_collection):
    # Calculate the average LAI and FAPAR within each feature (geometry) in the feature_collection
    average_lai = lai_image.reduceRegions(feature_collection, 
                                         ee.Reducer.mean(), 
                                         crs=lai_crs, crsTransform=lai_crsTransform, 
                                         tileScale=8)
    # Drop geometry information to keep only the properties
    average_lai = average_lai.map(lambda x: x.select(x.propertyNames(), retainGeometry=False))
    # Return the feature collection with average LAI and FAPAR information
    return ee.FeatureCollection(average_lai)

# Define a function to convert the wide-format output to long-format
def convert_wide_to_long(feature):
    # Define a function to select the LAI or FAPAR value for a specific band (date and unit)
    def select_band(band_name):
        band_name = ee.String(band_name)
        out_feature = ee.Feature(None, {})
        out_feature = out_feature.copyProperties(feature, management_units_properties)

        value = 0
        value = feature.get(band_name)

        band_name_split = band_name.split('_')
        year = ee.Number.parse(band_name_split.get(0))
        month = ee.Number.parse(band_name_split.get(1))
        unit = band_name_split.get(2)
        day = 1
        date = ee.Date.fromYMD(year, month, day)
        date = date.format('YYYY-MM-dd')

        out_feature = out_feature.set('Value', value, 'Year', year, 'Month', month, 'Day', day, 'Date', date, 'Unit', unit)
        return out_feature

    # Map the select_band function over the band names (dates and units) to create a new FeatureCollection
    output_collection = lai_image.bandNames().map(select_band)
    return ee.FeatureCollection(output_collection)

# Calculate average LAI and FAPAR for each management unit in the input feature collection
output = calculate_average_lai(management_units)

# Convert the output from wide to long format
output = output.map(convert_wide_to_long).flatten()

# Export the results to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection=output,
    description=output_name,
    fileNamePrefix=output_name,
    folder=output_folder)

# Start the export task
export_results_task.start()


In [9]:
# Get histogram of land cover values from South Africa National Land Cover Dataset
# https://egis.environment.gov.za/sa_national_land_cover_datasets

# Define output name and folder for the exported data
output_name = 'GCBR_SANLC_landcover_percentage'
output_folder = 'DOB_Ecology/GCBR'

# load land cover collection we uploaded to Google Earth Engine
landcover_collection = ee.ImageCollection('projects/resource-watch-gee/LocalDatasets/GCBR/loc_gcbr_006b_landcover_SALCC1')

# get projection information
landcover_scale = landcover_collection.first().projection().nominalScale()
landcover_projection = landcover_collection.first().projection().getInfo()
landcover_projection_gee = landcover_collection.first().projection()
landcover_crs = landcover_projection.get('crs')
landcover_crsTransform = landcover_projection.get('transform')

# define dictionary of land cover values and classes
landcover_values = np.arange(1,10).astype(str).tolist()
num_landcover_values = len(landcover_values)*[0]
empty_dictionary = dict(zip(landcover_values, num_landcover_values))
empty_dictionary = ee.Dictionary(empty_dictionary)

landcover_value_names = ["Forested land","Shrubland","Grassland","Waterbodies","Wetlands",
                         "Barren Land","Cultivated","Built-up","Mines & Quarries"]
landcover_name_dictionary = ee.Dictionary(dict(zip(landcover_values,landcover_value_names)))
print(landcover_name_dictionary.getInfo())


# Function to set histogram values and convert them to percentages
def set_histogram_values(feature):
    feature = ee.Feature(feature)
    new_histogram = ee.Dictionary(feature.get('histogram'))
    old_histogram = empty_dictionary
    histogram = old_histogram.combine(new_histogram, True)
    histogram_sum = ee.Array(histogram.values()).reduce(ee.Reducer.sum(), [0]).get([0])

    def convert_to_percentage(k, v):
        return ee.Number(v).divide(histogram_sum).multiply(100)

    new_dict = histogram.map(convert_to_percentage)
    new_dict = new_dict.rename(landcover_name_dictionary.keys(), landcover_name_dictionary.values())

    feature = feature.set(new_dict)
    feature = feature.select(feature.propertyNames().remove('histogram'))
    return feature

# Function to calculate the land cover histogram for each image in the collection
def calculate_landcover(image):
    lc_histogram = image.reduceRegions(management_units, reducer=ee.Reducer.frequencyHistogram(),
                                       crs=landcover_crs, crsTransform=landcover_crsTransform)
    year = ee.Number.parse(ee.String(image.get('system:index')).split('_').get(-2))
    date = ee.Date.fromYMD(year, 1, 1)
    lc_histogram = lc_histogram.map(lambda x: x.set({'date': date, 'year': year,'unit':'percentage'}))
    lc_histogram = lc_histogram.map(lambda x: x.select(x.propertyNames(), retainGeometry=False))
    return lc_histogram

# Map the 'empty_dictionary' to each feature in 'management_units' to add the dictionary properties to each feature
management_units_edit = management_units.map(lambda x: x.set(empty_dictionary))

# Map the 'calculate_landcover' function over the land cover collection and flatten the resulting collection
management_units_edit = landcover_collection.map(calculate_landcover).flatten()

# Map the 'set_histogram_values' function over 'management_units_edit' to set the histogram values and convert them to percentages
management_units_edit = management_units_edit.map(set_histogram_values)

# Export the results to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection=management_units_edit,
    description=output_name,
    fileNamePrefix=output_name,
    folder=output_folder)

export_results_task.start()

{'1': 'Forested land', '2': 'Shrubland', '3': 'Grassland', '4': 'Waterbodies', '5': 'Wetlands', '6': 'Barren Land', '7': 'Cultivated', '8': 'Built-up', '9': 'Mines & Quarries'}


In [10]:
# Get a histogram of surface water transitions from the JRC Global Surface Water dataset
# https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_GlobalSurfaceWater

# Define output name and folder for the exported data
output_name = 'GCBR_JRC_surface_water_transitions'
output_folder = 'DOB_Ecology/GCBR'

# Load the water transition collection from the JRC/GSW1_4/GlobalSurfaceWater dataset
water_transition_collection = ee.ImageCollection([ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('transition')])

# Get projection information from the first image in the water transition collection
water_transition_scale = water_transition_collection.first().projection().nominalScale()
water_transition_projection = water_transition_collection.first().projection().getInfo()
water_transition_projection_gee = water_transition_collection.first().projection()
water_transition_crs = water_transition_projection.get('crs')
water_transition_crsTransform = water_transition_projection.get('transform')

# Create a dictionary to store water transition names and their corresponding values
water_transition_values = np.arange(0, 11).astype(str).tolist()
num_water_transition_values = len(water_transition_values) * [0]
empty_dictionary = dict(zip(water_transition_values, num_water_transition_values))
empty_dictionary = ee.Dictionary(empty_dictionary)

# Define names for water transition values
water_transition_value_names = ['No change',
                                'Permanent',
                                'New Permanent',
                                'Lost Permanent',
                                'Seasonal',
                                'New Seasonal',
                                'Lost Seasonal',
                                'Seasonal to Permanent',
                                'Permanent to Seasonal',
                                'Ephemeral Permanent',
                                'Ephemeral Seasonal']
water_transition_name_dictionary = ee.Dictionary(dict(zip(water_transition_values, water_transition_value_names)))

# Define a function to set histogram values in each feature (a feature is like a single row in a table)
def set_histogram_values(feature):
    feature = ee.Feature(feature)
    new_histogram = ee.Dictionary(feature.get('histogram'))
    old_histogram = empty_dictionary
    histogram = old_histogram.combine(new_histogram, True)
    histogram_sum = ee.Array(histogram.values()).reduce(ee.Reducer.sum(), [0]).get([0])

    def convert_to_percentage(k, v):
        return ee.Number(v).divide(histogram_sum).multiply(100)

    new_dict = histogram.map(convert_to_percentage)
    new_dict = new_dict.rename(water_transition_name_dictionary.keys(), water_transition_name_dictionary.values())

    feature = feature.set(new_dict)
    feature = feature.set({'unit':'percentage'})
    feature = feature.select(feature.propertyNames().remove('histogram'))
    return feature

# Define a function to calculate water transition histograms for each image
def calculate_water_transition(image):
    water_transition_histogram = image.reduceRegions(management_units, reducer=ee.Reducer.frequencyHistogram(), 
                                                    crs=water_transition_crs, crsTransform=water_transition_crsTransform)
    
    # Select all properties except geometry to remove duplicates
    water_transition_histogram = water_transition_histogram.map(lambda x: x.select(x.propertyNames(), retainGeometry=False))
    return water_transition_histogram

# Create a copy of management_units with an empty dictionary as a property
management_units_edit = management_units.map(lambda x: x.set(empty_dictionary))

# Calculate water transition histograms for each image in the water_transition_collection
management_units_edit = water_transition_collection.map(calculate_water_transition).flatten()

# Set histogram values in the management_units_edit features
management_units_edit = management_units_edit.map(set_histogram_values)

# Export the results to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection=management_units_edit,
    description=output_name,
    fileNamePrefix=output_name,
    folder=output_folder)

# Start the export task
export_results_task.start()


In [11]:
# Calculate monthly average rainfall from the CHIRPS pentad (5-day) precipitation data 
# https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD

# Define output name and folder for the exported data
output_name = 'GCBR_CHIRPS_average_rainfall'
output_folder = 'DOB_Ecology/GCBR'

# Load the CHIRPS pentad (5-day) precipitation data from the UCSB-CHG/CHIRPS/PENTAD dataset
chirps_collection = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")

# Get projection information from the first image in the CHIRPS collection
chirps_projection = chirps_collection.first().projection().getInfo()
chirps_projection_gee = chirps_collection.first().projection()
chirps_crs = chirps_projection.get('crs')
chirps_crsTransform = chirps_projection.get('transform')

# Define lists of months and years
months = ee.List([str(x) for x in np.arange(1, 13)])
years = ee.List([str(x) for x in np.arange(2000, 2021)])

# Define a function to filter the CHIRPS collection for a specific year and month and calculate the sum of precipitation
def filter_year(y):
    def filter_month(m):
        year = ee.Number.parse(y)
        month = ee.Number.parse(m)
        new_image = chirps_collection.filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(month, month, 'month')).sum()
        new_image = new_image.set('system:index', ee.String(y).cat(ee.String('_')).cat(ee.String(m)))
        return new_image
    return months.map(filter_month)

# Create a new ImageCollection by mapping the filter_year function over the years and months
chirps_collection = ee.ImageCollection.fromImages(years.map(filter_year).flatten())

# Convert the ImageCollection to a single multiband image
chirps_image = chirps_collection.toBands()

# Define a function to calculate CHIRPS rainfall for each feature in the input feature collection
def calculate_chirps_rainfall(feature_collection):
    # Calculate average rainfall within each feature (geometry) in the feature_collection
    sum_rainfall = chirps_image.reduceRegions(feature_collection, 
                                              ee.Reducer.mean(), 
                                              crs=chirps_crs, crsTransform=chirps_crsTransform, 
                                              tileScale=8)
    # Drop geometry information to keep only the properties
    sum_rainfall = sum_rainfall.map(lambda x: x.select(x.propertyNames(), retainGeometry=False))
    # Return the feature collection with rainfall information
    return ee.FeatureCollection(sum_rainfall)

# Define a function to convert the wide-format output to long-format
def convert_wide_to_long(feature):
    # Define a function to select the precipitation value for a specific band (date)
    def select_band(band_name):
        band_name = ee.String(band_name)
        out_feature = ee.Feature(None, {})
        out_feature = out_feature.copyProperties(feature, management_units_properties)

        value = 0
        value = feature.get(band_name)

        band_name_split = band_name.split('_')
        year = ee.Number.parse(band_name_split.get(0))
        month = ee.Number.parse(band_name_split.get(1))
        day = 1
        date = ee.Date.fromYMD(year, month, day)
        date = date.format('YYYY-MM-dd')

        out_feature = out_feature.set('Value', value, 'Year', year, 'Month', month, 'Day', day, 'Date', date, 'Unit', 'millimeters')
        return out_feature

    # Map the select_band function over the band names (dates) to create a new FeatureCollection
    output_collection = chirps_image.bandNames().map(select_band)
    return ee.FeatureCollection(output_collection)


# Calculate CHIRPS rainfall for each management unit in the input feature collection
output = calculate_chirps_rainfall(management_units)

# Convert the output from wide to long format
output = output.map(convert_wide_to_long).flatten()

# Export the results to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection=output,
    description=output_name,
    fileNamePrefix=output_name,
    folder=output_folder)

# Start the export task
export_results_task.start()
